In [1]:
import os
import librosa
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import librosa.display

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [2]:
# get the parent directory
parent_dir = os.path.abspath('..')
test_dir = os.path.join(parent_dir, 'Testing')
data_dir = os.path.join(parent_dir, 'Data')
meta_dir = os.path.join(data_dir, 'metadata')
sour_dir = os.path.join(data_dir, 'source')
data_dir = os.path.join(parent_dir, 'EDA')

In [3]:
sour_dir

'C:\\Users\\aubin\\beCode\\anomaly\\AudioMalfunctionDetection\\Data\\source'

In [61]:
def get_melspectogram(file_path, window, no_channel=None, overlap=None, n_fft=None, n_mels=32, machine=None):
    
    # Load the signal
    signal, fs = librosa.load(file_path, sr=None, mono=False)
    
    # Default channel no selected based on machine
    channel_dict = {'pump': 3, 'valve': 1, 'slider': 7, 'fan': 5}
    if no_channel:
        signal = signal[no_channel, :]
    else:
        signal = signal.mean(axis=0)
    
    # Window length in samples
    window_length = int(window*fs)
    
    # Default overlap is 50% of the window size
    if overlap==None:
        overlap = 0.5
    
    # Overlap in samples
    overlap_length = int(window_length*overlap)
    
    # Hop length in samples
    hop_length = window_length-overlap_length
    
    # Default n_fft is the smallest power of 2 larger than win_length
    if n_fft==None:
        n_fft = int(2**np.ceil(np.log2(window_length)))
   
    # Compute mel spectogram
    mel_spect = librosa.feature.melspectrogram(y=signal, sr=fs, 
                                               win_length=window_length, 
                                               hop_length=hop_length,
                                               n_fft=n_fft,
                                               n_mels=n_mels)
    # Mel spectogram in decibels
    mel_spect_db = librosa.power_to_db(mel_spect, ref=1.0, amin=sys.float_info.epsilon, top_db=np.inf)

    params = {}
    params['window'] = window
    params['window_length'] = window_length
    params['overlap'] = overlap
    params['overlap_length'] = overlap_length
    params['hop_length'] = hop_length
    params['n_fft'] = n_fft
    params['n_mels'] = n_mels
    
    return mel_spect, mel_spect_db, params

In [62]:
fan_test_set = pd.read_csv(os.path.join(test_dir, 'fan_subset.csv'))

In [6]:
fan_test_set.iloc[0]['file_rel_path']

'6_dB_fan\\fan\\id_00\\abnormal\\00000020.wav'

In [7]:
mel_spect, mel_spect_db, params = get_melspectogram(os.path.join(sour_dir, fan_test_set.iloc[0]['file_rel_path']), window = 1)

In [35]:
all_mel_spect = []
all_mel_spect_db = []
all_params = []

for idx, row in fan_test_set.iterrows():
    mel_spect, mel_spect_db, params = get_melspectogram(os.path.join(sour_dir, row['file_rel_path']), window = 1)
    all_mel_spect.append(mel_spect)
    all_mel_spect_db.append(mel_spect_db)
    all_params.append(params)

In [36]:
fan_test_set['mel_spect'] = all_mel_spect
fan_test_set['mel_spect_db'] = all_mel_spect_db
fan_test_set['all_params'] = all_params

In [37]:
fan_test_set

,file_no,machine,model,anomaly,file_name,file_rel_path,duration_s,fs_Hz,SNR_dB,mel_spect,mel_spect_db,all_params
0,20,fan,0,1,00000020.wav,6_dB_fan\fan\id_00\abnormal\00000020.wav,10.0,16000,6,"[[24.14678, 25.470774, 33.14535, 32.042454, 34...","[[13.828592, 14.060421, 15.204226, 15.057258, ...","{'window': 1, 'window_length': 16000, 'overlap..."
1,194,fan,0,1,00000194.wav,6_dB_fan\fan\id_00\abnormal\00000194.wav,10.0,16000,6,"[[2.2732325, 4.6159825, 5.5582514, 4.1978726, ...","[[3.5664387, 6.6426415, 7.449382, 6.2302923, 6...","{'window': 1, 'window_length': 16000, 'overlap..."
2,55,fan,0,1,00000055.wav,6_dB_fan\fan\id_00\abnormal\00000055.wav,10.0,16000,6,"[[24.730875, 30.767958, 38.945984, 25.486095, ...","[[13.932395, 14.880986, 15.904627, 14.063033, ...","{'window': 1, 'window_length': 16000, 'overlap..."
3,204,fan,0,1,00000204.wav,6_dB_fan\fan\id_00\abnormal\00000204.wav,10.0,16000,6,"[[6.9695573, 3.3872268, 3.9149032, 3.9909353, ...","[[8.432053, 5.298443, 5.9272103, 6.010747, 4.8...","{'window': 1, 'window_length': 16000, 'overlap..."
4,173,fan,0,1,00000173.wav,6_dB_fan\fan\id_00\abnormal\00000173.wav,10.0,16000,6,"[[2.6273892, 2.0992374, 2.7908661, 2.8017392, ...","[[4.195244, 3.2206156, 4.45739, 4.474277, 6.71...","{'window': 1, 'window_length': 16000, 'overlap..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1595,4712,fan,6,0,00000177.wav,6_dB_fan\fan\id_06\normal\00000177.wav,10.0,16000,6,"[[0.3284199, 0.4780445, 0.5127091, 0.37700543,...","[[-4.8357053, -3.2053165, -2.9012897, -4.23652...","{'window': 1, 'window_length': 16000, 'overlap..."
1596,4810,fan,6,0,00000275.wav,6_dB_fan\fan\id_06\normal\00000275.wav,10.0,16000,6,"[[0.19321513, 0.23406115, 0.16147715, 0.138021...","[[-7.1395884, -6.3067064, -7.9188895, -8.60052...","{'window': 1, 'window_length': 16000, 'overlap..."
1597,5100,fan,6,0,00000565.wav,6_dB_fan\fan\id_06\normal\00000565.wav,10.0,16000,6,"[[1.4041063, 2.2102797, 3.0896509, 1.9302347, ...","[[1.4739997, 3.4444723, 4.899094, 2.856101, -0...","{'window': 1, 'window_length': 16000, 'overlap..."
1598,4548,fan,6,0,00000013.wav,6_dB_fan\fan\id_06\normal\00000013.wav,10.0,16000,6,"[[0.7297915, 0.9056003, 1.2177532, 1.0107999, ...","[[-1.3680118, -0.43063438, 0.8555927, 0.046651...","{'window': 1, 'window_length': 16000, 'overlap..."


In [39]:
X = []

for idx, row in fan_test_set.iterrows():
    #X.append(row['mel_spect_db'].reshape(1, total_size))
    X.append(np.array(row['mel_spect_db']).flatten())

In [43]:
len(y_train)

1280

In [8]:
y = fan_test_set['anomaly']

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

In [44]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train,y_train)

predicted= model.predict(X_test) 

In [45]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, predicted))

Accuracy: 0.953125


In [133]:
# optimizing : 
#running through all the parameters and selecting the highest ones.

#best number of mels 64, best window size, 1.0, best channel = 3, best number of neighbours = 5

In [55]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

In [69]:
results_knn = {}

results_knn['window'] = []
results_knn['window_length'] = []
results_knn['overlap'] = []
results_knn['overlap_length'] = []
results_knn['hop_length'] = []
results_knn['n_fft'] = []
results_knn['n_mels'] = []
results_knn['n_neighbor'] = []
results_knn['channel'] = []
results_knn['accuracy'] = []
results_knn['recall'] = []
results_knn['precision'] = []
results_knn['f1'] = []
results_knn['cross_val_scores'] = []
results_knn['cross_val_mean'] = []
results_knn['cross_val_stdev'] = []

for x in range(2):
    all_mel_spect = []
    all_mel_spect_db = []
    all_params = []
    
    channel_p = 3
    n_neighbors_p = 5
    window_p = 1
    overlap_p = None
    n_fft_p = None
    n_mels_p = 32

    for idx, row in fan_test_set.iterrows():
        mel_spect, mel_spect_db, params = get_melspectogram(
            os.path.join(sour_dir, row['file_rel_path']), 
            no_channel = channel_p, 
            window = window_p,
            overlap = overlap_p, 
            n_fft = n_fft_p, 
            n_mels = n_mels_p
        )
        
        all_mel_spect.append(mel_spect)
        all_mel_spect_db.append(mel_spect_db)
        # add the number of params to the stored params
        params['n_neighbor'] = n_neighbors_p
        params['channel'] = channel_p
        all_params.append(params)
        
    fan_test_set['mel_spect'] = all_mel_spect
    fan_test_set['mel_spect_db'] = all_mel_spect_db
    fan_test_set['all_params'] = all_params
        
    y = fan_test_set['anomaly']
    X = []
    for idx, row in fan_test_set.iterrows():
        #X.append(row['mel_spect_db'].reshape(1, total_size))
        X.append(np.array(row['mel_spect_db']).flatten())

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)
        
    model = KNeighborsClassifier(n_neighbors=n_neighbors_p)
    model.fit(X_train,y_train)

    predicted= model.predict(X_test) 

    # calculate results
    accuracy = metrics.accuracy_score(y_test, predicted)
    recall = metrics.recall_score(predicted, y_test, average='macro')
    precision = metrics.precision_score(predicted, y_test, average='macro')
    f1 = metrics.f1_score(predicted, y_test, average='macro')
    
    # cross-validate
    cross_val_scores = cross_val_score(model, X, y, cv=5) # cv is the number of folds (k)
    cross_val_mean = cross_val_scores.mean() * 100 
    cross_val_stdev = cross_val_scores.std() * 100

    for key, val in params.items():
        results_knn[key].append(val)
    results_knn['accuracy'].append(accuracy)
    results_knn['recall'].append(recall)
    results_knn['precision'].append(precision)
    results_knn['f1'].append(f1)
    results_knn['cross_val_scores'].append(cross_val_scores)
    results_knn['cross_val_mean'].append(cross_val_mean)
    results_knn['cross_val_stdev'].append(cross_val_stdev)
    
    print(f"params: {params}")
    print(f"Accuracy: {accuracy}")
    print(f"recall: {recall}")
    print(f"precision: {precision}")
    print(f"f1: {f1}")
    print("cross_val mean and acc: {:.2f}% (+/- {:.2f})".format(cross_val_mean, cross_val_stdev))
        

params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 32, 'n_neighbor': 1, 'channel': 3}
Accuracy: 0.9875
recall: 0.9850292397660819
precision: 0.9850292397660819
f1: 0.9850292397660819
cross_val mean and acc: 96.06% (+/- 6.22)
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 32, 'n_neighbor': 3, 'channel': 3}
Accuracy: 0.9875
recall: 0.9850292397660819
precision: 0.9850292397660819
f1: 0.9850292397660819
cross_val mean and acc: 95.50% (+/- 7.94)
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 32, 'n_neighbor': 5, 'channel': 3}
Accuracy: 0.9875
recall: 0.9850292397660819
precision: 0.9850292397660819
f1: 0.9850292397660819
cross_val mean and acc: 95.00% (+/- 8.18)
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 80

In [70]:
results = pd.DataFrame(results_knn)

In [71]:
results.to_csv('no_neighbours.csv')

In [73]:
results_knn = {}

results_knn['window'] = []
results_knn['window_length'] = []
results_knn['overlap'] = []
results_knn['overlap_length'] = []
results_knn['hop_length'] = []
results_knn['n_fft'] = []
results_knn['n_mels'] = []
results_knn['n_neighbor'] = []
results_knn['channel'] = []
results_knn['accuracy'] = []
results_knn['recall'] = []
results_knn['precision'] = []
results_knn['f1'] = []
results_knn['cross_val_scores'] = []
results_knn['cross_val_mean'] = []
results_knn['cross_val_stdev'] = []

model_types = ['fan_m1_subset.csv', 'fan_m2_subset.csv', 'fan_m3_subset.csv', 'fan_m4_subset.csv']

for x in model_types:
    fan_test_set = pd.read_csv(os.path.join(test_dir, x))
    
    all_mel_spect = []
    all_mel_spect_db = []
    all_params = []
    
    channel_p = 3
    n_neighbors_p = 5
    window_p = 1
    overlap_p = None
    n_fft_p = None
    n_mels_p = 32

    for idx, row in fan_test_set.iterrows():
        mel_spect, mel_spect_db, params = get_melspectogram(
            os.path.join(sour_dir, row['file_rel_path']), 
            no_channel = channel_p, 
            window = window_p,
            overlap = overlap_p, 
            n_fft = n_fft_p, 
            n_mels = n_mels_p
        )
        
        all_mel_spect.append(mel_spect)
        all_mel_spect_db.append(mel_spect_db)
        # add the number of params to the stored params
        params['n_neighbor'] = n_neighbors_p
        params['channel'] = channel_p
        all_params.append(params)
        
    fan_test_set['mel_spect'] = all_mel_spect
    fan_test_set['mel_spect_db'] = all_mel_spect_db
    fan_test_set['all_params'] = all_params
        
    y = fan_test_set['anomaly']
    X = []
    for idx, row in fan_test_set.iterrows():
        #X.append(row['mel_spect_db'].reshape(1, total_size))
        X.append(np.array(row['mel_spect_db']).flatten())

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)
        
    model = KNeighborsClassifier(n_neighbors=n_neighbors_p)
    model.fit(X_train,y_train)

    predicted= model.predict(X_test) 

    # calculate results
    accuracy = metrics.accuracy_score(y_test, predicted)
    recall = metrics.recall_score(predicted, y_test, average='macro')
    precision = metrics.precision_score(predicted, y_test, average='macro')
    f1 = metrics.f1_score(predicted, y_test, average='macro')
    
    # cross-validate
    cross_val_scores = cross_val_score(model, X, y, cv=5) # cv is the number of folds (k)
    cross_val_mean = cross_val_scores.mean() * 100 
    cross_val_stdev = cross_val_scores.std() * 100

    for key, val in params.items():
        results_knn[key].append(val)
    results_knn['accuracy'].append(accuracy)
    results_knn['recall'].append(recall)
    results_knn['precision'].append(precision)
    results_knn['f1'].append(f1)
    results_knn['cross_val_scores'].append(cross_val_scores)
    results_knn['cross_val_mean'].append(cross_val_mean)
    results_knn['cross_val_stdev'].append(cross_val_stdev)
    
    print(f"model type : {x}")
    print(f"params: {params}")
    print(f"Accuracy: {accuracy}")
    print(f"recall: {recall}")
    print(f"precision: {precision}")
    print(f"f1: {f1}")
    print("cross_val mean and acc: {:.2f}% (+/- {:.2f})".format(cross_val_mean, cross_val_stdev))
        

model type : fan_m1_subset.csv
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 32, 'n_neighbor': 5, 'channel': 3}
Accuracy: 0.9375
recall: 0.9615384615384616
precision: 0.875
f1: 0.9085714285714286
cross_val mean and acc: 90.00% (+/- 2.96)
model type : fan_m2_subset.csv
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 32, 'n_neighbor': 5, 'channel': 3}
Accuracy: 1.0
recall: 1.0
precision: 1.0
f1: 1.0
cross_val mean and acc: 100.00% (+/- 0.00)
model type : fan_m3_subset.csv
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 32, 'n_neighbor': 5, 'channel': 3}
Accuracy: 1.0
recall: 1.0
precision: 1.0
f1: 1.0
cross_val mean and acc: 100.00% (+/- 0.00)
model type : fan_m4_subset.csv
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'ov

In [74]:
results = pd.DataFrame(results_knn)

In [75]:
results.to_csv('per_model_type.csv')

In [76]:
# working on the pump
# channel = 1, n_mels is 512 or maybe greater, no_neighbors = 3

In [86]:
results_knn = {}

results_knn['window'] = []
results_knn['window_length'] = []
results_knn['overlap'] = []
results_knn['overlap_length'] = []
results_knn['hop_length'] = []
results_knn['n_fft'] = []
results_knn['n_mels'] = []
results_knn['n_neighbor'] = []
results_knn['channel'] = []
results_knn['accuracy'] = []
results_knn['recall'] = []
results_knn['precision'] = []
results_knn['f1'] = []
results_knn['cross_val_scores'] = []
results_knn['cross_val_mean'] = []
results_knn['cross_val_stdev'] = []

model_types = ['pump_m1_subset.csv', 'pump_m2_subset.csv', 'pump_m3_subset.csv', 'pump_m4_subset.csv']

for x in model_types:
    fan_test_set = pd.read_csv(os.path.join(test_dir, x))
    
    all_mel_spect = []
    all_mel_spect_db = []
    all_params = []
    
    channel_p = 1
    n_neighbors_p = 3
    window_p = 1
    overlap_p = None
    n_fft_p = None
    n_mels_p = 512

    for idx, row in fan_test_set.iterrows():
        mel_spect, mel_spect_db, params = get_melspectogram(
            os.path.join(sour_dir, row['file_rel_path']), 
            no_channel = channel_p, 
            window = window_p,
            overlap = overlap_p, 
            n_fft = n_fft_p, 
            n_mels = n_mels_p
        )
        
        all_mel_spect.append(mel_spect)
        all_mel_spect_db.append(mel_spect_db)
        # add the number of params to the stored params
        params['n_neighbor'] = n_neighbors_p
        params['channel'] = channel_p
        all_params.append(params)
        
    fan_test_set['mel_spect'] = all_mel_spect
    fan_test_set['mel_spect_db'] = all_mel_spect_db
    fan_test_set['all_params'] = all_params
        
    y = fan_test_set['anomaly']
    X = []
    for idx, row in fan_test_set.iterrows():
        #X.append(row['mel_spect_db'].reshape(1, total_size))
        X.append(np.array(row['mel_spect_db']).flatten())

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)
        
    model = KNeighborsClassifier(n_neighbors=n_neighbors_p)
    model.fit(X_train,y_train)

    predicted= model.predict(X_test) 

    # calculate results
    accuracy = metrics.accuracy_score(y_test, predicted)
    recall = metrics.recall_score(predicted, y_test, average='macro')
    precision = metrics.precision_score(predicted, y_test, average='macro')
    f1 = metrics.f1_score(predicted, y_test, average='macro')
    
    # cross-validate
    cross_val_scores = cross_val_score(model, X, y, cv=5) # cv is the number of folds (k)
    cross_val_mean = cross_val_scores.mean() * 100 
    cross_val_stdev = cross_val_scores.std() * 100

    for key, val in params.items():
        results_knn[key].append(val)
    results_knn['accuracy'].append(accuracy)
    results_knn['recall'].append(recall)
    results_knn['precision'].append(precision)
    results_knn['f1'].append(f1)
    results_knn['cross_val_scores'].append(cross_val_scores)
    results_knn['cross_val_mean'].append(cross_val_mean)
    results_knn['cross_val_stdev'].append(cross_val_stdev)
    
    print(f"params: {params}")
    print(f"Accuracy: {accuracy}")
    print(f"recall: {recall}")
    print(f"precision: {precision}")
    print(f"f1: {f1}")
    print("cross_val mean and acc: {:.2f}% (+/- {:.2f})".format(cross_val_mean, cross_val_stdev))
        

params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 512, 'n_neighbor': 3, 'channel': 1}
Accuracy: 0.975
recall: 0.9838709677419355
precision: 0.95
f1: 0.9654874892148404
cross_val mean and acc: 99.00% (+/- 0.94)
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 512, 'n_neighbor': 3, 'channel': 1}
Accuracy: 1.0
recall: 1.0
precision: 1.0
f1: 1.0
cross_val mean and acc: 99.75% (+/- 0.50)
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 512, 'n_neighbor': 3, 'channel': 1}
Accuracy: 1.0
recall: 1.0
precision: 1.0
f1: 1.0
cross_val mean and acc: 99.75% (+/- 0.50)
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 512, 'n_neighbor': 3, 'channel': 1}
Accuracy: 1.0
recall: 

In [87]:
results = pd.DataFrame(results_knn)
results.to_csv('pump_per_model.csv')

In [88]:
# slide_rail

# channel 6!!, n_mels 512, 

In [92]:
results_knn = {}

results_knn['window'] = []
results_knn['window_length'] = []
results_knn['overlap'] = []
results_knn['overlap_length'] = []
results_knn['hop_length'] = []
results_knn['n_fft'] = []
results_knn['n_mels'] = []
results_knn['n_neighbor'] = []
results_knn['channel'] = []
results_knn['accuracy'] = []
results_knn['recall'] = []
results_knn['precision'] = []
results_knn['f1'] = []
results_knn['cross_val_scores'] = []
results_knn['cross_val_mean'] = []
results_knn['cross_val_stdev'] = []

model_types = ['pump_m1_subset.csv', 'pump_m2_subset.csv', 'pump_m3_subset.csv', 'pump_m4_subset.csv']

for x in [8,16,32,64,128,216,532]:
    fan_test_set = pd.read_csv(os.path.join(test_dir, 'slider_subset.csv'))
    
    all_mel_spect = []
    all_mel_spect_db = []
    all_params = []
    
    channel_p = 6
    n_neighbors_p = 3
    window_p = 1
    overlap_p = None
    n_fft_p = None
    n_mels_p = x

    for idx, row in fan_test_set.iterrows():
        mel_spect, mel_spect_db, params = get_melspectogram(
            os.path.join(sour_dir, row['file_rel_path']), 
            no_channel = channel_p, 
            window = window_p,
            overlap = overlap_p, 
            n_fft = n_fft_p, 
            n_mels = n_mels_p
        )
        
        all_mel_spect.append(mel_spect)
        all_mel_spect_db.append(mel_spect_db)
        # add the number of params to the stored params
        params['n_neighbor'] = n_neighbors_p
        params['channel'] = channel_p
        all_params.append(params)
        
    fan_test_set['mel_spect'] = all_mel_spect
    fan_test_set['mel_spect_db'] = all_mel_spect_db
    fan_test_set['all_params'] = all_params
        
    y = fan_test_set['anomaly']
    X = []
    for idx, row in fan_test_set.iterrows():
        #X.append(row['mel_spect_db'].reshape(1, total_size))
        X.append(np.array(row['mel_spect_db']).flatten())

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)
        
    model = KNeighborsClassifier(n_neighbors=n_neighbors_p)
    model.fit(X_train,y_train)

    predicted= model.predict(X_test) 

    # calculate results
    accuracy = metrics.accuracy_score(y_test, predicted)
    recall = metrics.recall_score(predicted, y_test, average='macro')
    precision = metrics.precision_score(predicted, y_test, average='macro')
    f1 = metrics.f1_score(predicted, y_test, average='macro')
    
    # cross-validate
    cross_val_scores = cross_val_score(model, X, y, cv=5) # cv is the number of folds (k)
    cross_val_mean = cross_val_scores.mean() * 100 
    cross_val_stdev = cross_val_scores.std() * 100

    for key, val in params.items():
        results_knn[key].append(val)
    results_knn['accuracy'].append(accuracy)
    results_knn['recall'].append(recall)
    results_knn['precision'].append(precision)
    results_knn['f1'].append(f1)
    results_knn['cross_val_scores'].append(cross_val_scores)
    results_knn['cross_val_mean'].append(cross_val_mean)
    results_knn['cross_val_stdev'].append(cross_val_stdev)
    
    print(f"params: {params}")
    print(f"Accuracy: {accuracy}")
    print(f"recall: {recall}")
    print(f"precision: {precision}")
    print(f"f1: {f1}")
    print("cross_val mean and acc: {:.2f}% (+/- {:.2f})".format(cross_val_mean, cross_val_stdev))
        

params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 8, 'n_neighbor': 3, 'channel': 6}
Accuracy: 0.8291666666666667
recall: 0.7959508912761185
precision: 0.7271656397245378
f1: 0.750095238095238
cross_val mean and acc: 85.25% (+/- 10.85)
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 16, 'n_neighbor': 3, 'channel': 6}
Accuracy: 0.8375
recall: 0.8101168693188525
precision: 0.7380391446176151
f1: 0.7622857142857142
cross_val mean and acc: 85.67% (+/- 11.34)
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_length': 8000, 'hop_length': 8000, 'n_fft': 16384, 'n_mels': 32, 'n_neighbor': 3, 'channel': 6}
Accuracy: 0.8625
recall: 0.847008547008547
precision: 0.775915186661834
f1: 0.8015584675903886
cross_val mean and acc: 87.00% (+/- 10.29)
params: {'window': 1, 'window_length': 16000, 'overlap': 0.5, 'overlap_

In [93]:
results = pd.DataFrame(results_knn)
results.to_csv('slider_no_mels.csv')